###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2022  by D. Koehn, T. Meier and J. Stampa, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Digitale Signalverarbeitung: Signaldetektion

## Kapitel 6 Übungen

### Aufgabe 6.1 

Schreiben Sie die Formeln für die rekursive Berechnung des STA/LTA-Detektors auf. Verwenden Sie die im Abschnitt 4.5.2 Moving Average dargestellten Überlegungen. 

Laut Gl. (4.21) definieren wir das einseitige, gleitende Mittel mit einer Fensterlänge $q$ als

\begin{equation}
\hat{m}_i = \frac{1}{q+1} \sum_{j=0}^q x_{i-j} \tag{4.21}
\end{equation}

Die gleitende Mittel für das Sample $i$ läßt sich rekursiv aus dem gleitenden Mittel für das Sample $i-1$ berechnen:

\begin{equation}
\hat{m}_{i} = \hat{m}_{i-1} - \frac{1}{q+1} x_{i-q-1} + \frac{1}{q+1} x_{i} \tag{4.22}
\end{equation}

Setzt man $x_{i-q-1} := \hat{m}_{i-1}$ und definiert $\alpha := \frac{1}{q+1}$, ergibt sich

\begin{equation}
\hat{m}_{i} = \hat{m}_{i-1} (1 - \alpha) + \alpha x_{i} \tag{4.23}
\end{equation}

Wie wir später bei der Berechnung von STA, LTA und Kurtosis sehen werden, stabilisiert diese Näherung die Ergebnisse der rekursiven Berechnung.

Analog lassen sich rekursive Formeln für den $\text{STA}$ und $\text{LTA}$ Wert berechnen. $\text{STA}$ und $\text{LTA}$ sind definiert als:

\begin{equation}
\text{STA}_i = \frac{1}{q_{\text{STA}}+1} \sum_{j=0}^{q_{\text{STA}}} |x_{i-j}| \notag
\end{equation}

\begin{equation}
\text{LTA}_i = \frac{1}{q_{\text{LTA}}+1} \sum_{j=0}^{q_{\text{LTA}}} |x_{i-j}| \notag
\end{equation}

Für die rekursive Berechnung ergibt sich analog zum gleitenden Mittel:

\begin{equation}
\text{STA}_{i} = \text{STA}_{i-1} - \frac{1}{q_\text{STA}+1} |x_{i-q_\text{STA}-1}| + \frac{1}{q_\text{STA}+1} |x_{i}| \notag
\end{equation}

\begin{equation}
\text{LTA}_{i} = \text{LTA}_{i-1} - \frac{1}{q_\text{LTA}+1} |x_{i-q_\text{LTA}-1}| + \frac{1}{q_\text{LTA}+1} |x_{i}| \notag
\end{equation}

oder 

\begin{equation}
\text{STA}_{i} = \text{STA}_{i-1} (1 - \alpha_{\text{STA}}) + \alpha_{\text{STA}} |x_{i}| \notag
\end{equation}

\begin{equation}
\text{LTA}_{i} = \text{LTA}_{i-1} (1 - \alpha_{\text{LTA}}) + \alpha_{\text{LTA}} |x_{i}| \notag
\end{equation}

mit $\alpha_{\text{STA}} = \frac{1}{q_\text{STA}+1}$ und $\alpha_{\text{LTA}} = \frac{1}{q_\text{LTA}+1}$.

Nachdem STA und LTA rekursiv berechnet wurden, folgt für die charakteristische Funktion:

\begin{equation}
\text{CF}_i = \frac{\text{STA}_{i}}{\text{LTA}_{i}} \notag
\end{equation}

### Aufgabe 6.2

Schreiben Sie die Formeln für die rekursive Berechung der Kurtosis im gleitenden Fenster auf.

Das zentrale nte-Moment läßt sich laut Gl. (2.40) folgendermaßen abschätzen:

\begin{equation}
\hat{\alpha}_i^{(n)} = \frac{1}{q_K} \sum_{j=0}^{q_K} (x_{i-j}-\hat{m}_i)^n \notag
\end{equation}

wobei zuvor das gleitende Mittel berechnet werden muß.

Die rekursive Berechnung ergibt:

\begin{equation}
\hat{\alpha}_{i}^{(n)} = \hat{\alpha}_{i-1}^{(n)} - \frac{1}{q_K} (x_{i-q_K-1}-\hat{m}_{i-1})^n + \frac{1}{q_K} (x_{i}-\hat{m}_{i})^n \notag
\end{equation}

oder mit der Näherung $(x_{i-q_K-1}-\hat{m}_{i-1})^n := \hat{\alpha}_{i-1}^{(n)}$ und $ \alpha_K := \frac{1}{q_K}$ folgt:

\begin{equation}
\hat{\alpha}_{i}^{(n)} = \hat{\alpha}_{i-1}^{(n)} (1 - \alpha_K) + \alpha_K (x_{i}-\hat{m}_{i})^n \notag
\end{equation}

Nach der Berechnung von $\hat{\alpha}_{i}^{(4)}$ und $\hat{\alpha}_{i}^{(2)}$ folgt die Kurtosis zu:

\begin{equation}
K_i = \frac{\hat{\alpha}_{i}^{(4)}}{(\hat{\alpha}_{i}^{(2)})^2} \notag
\end{equation}

In [2]:
# Importiere Python Bibliotheken 
# ------------------------------
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

### Aufgabe 6.3 & 6.4

Definieren Sie eine Wertereihe, die aus einem Impuls und zufälligem normalverteiltem Rauschen bestehen soll. Wenden Sie den STA/LTA-Detektor auf diese synthetische Wertereihe an. Finden Sie optimale Parameter für die Länge des STA bzw. LTA.

Berechnen Sie die charakteristische Funktion mittels Schätzung der Kurtosis im gleitenden Fenster. Vergleichen Sie das Ergebnis mit der charakteristischen Funktion des STA/LTA-Detektors.

Die Implementierung der rekursiven Algorithmen ist relativ einfach. Man berechnet zunächst die entsprechenden Werte für das erste Sample und wendet anschließend die Rekursion an. 

Wichtig ist, daß man vor Beginn der Zeitreihe nicht mit Nullen zeropadden darf, da sonst der Beginn der Zeitreihe als Signal detektiert wird. Eine Möglichkeit das Problem zu lösen ist den Beginn der Zeitreihe mit einer gespiegelten Version der Zeitreihe, die man auf die Fensterlänge zuschneidet zu "zeropadden".

Da man bei der Berechnung von STA/LTA und Kurtosis teilweise durch kleine Werte teilt, sollte man auch eine Regularisierung des Nenners implementieren.

In [3]:
def stalta_kurtosis(sn, wsta, wlta, wk, kurtosis, wavelet):
    
    # Define parameters
    dt = 1.                 # time sampling [s]
    L = 800.                # length of the time series [s]

    # Define time array
    t = np.arange(0,L+dt,dt) # compute time vector
    N = len(t)
    
    #  Add spike impulses to time series x ...
    x = np.zeros(N)
    x[N//2] = 1.
    x[N//3] = .75
    x[N//4] = .5
    
    # Convolve source wavelet with spike response
    if(wavelet==True):
        
        wc = 2. * np.pi / 4.
        src = np.sin(wc*t) * np. exp(-1e-1*t)
        x = np.convolve(src,x)
        x = x[:N]
    
    # ... create normal random noise with standard deviation sn ...
    # Here I set a fixed seed to reproduce the same random numbers every time 
    # I change the parameters sn, wsta, wlta, wk, kurtosis, so the time series 
    # does not change. THIS IS A VERY BAD PRACTICE: DON'T DO THIS IF
    # IF YOUR APPLICATION HAS TO RELY ON RANDOM NUMBERS
    np.random.seed(seed=12345656)
    xnoise = np.random.normal(0., sn, N)
    
    # ... add random noise to trend
    x = x + xnoise
    
    # compute window lengths from [s] -> #samples
    q_sta = (int)(wsta//dt)
    q_lta = (int)(wlta//dt)
    q_k = (int)(wk//dt)
    
    # pad beginning of time series with time reversed time series
    x1 = np.concatenate((np.flipud(x[:q_lta]), x), axis=0) 
    N1 = len(x1)
    
    # compute alpha factors 
    alpha_sta = 1. / (q_sta+1)
    alpha_lta = 1. / (q_lta+1)
    alpha_k = 1. / (q_k)
    
    # compute STA and LTA values 
    # --------------------------
    
    # initialize STA and LTA values
    STA = np.zeros(N)
    LTA = np.zeros(N)
    
    # compute first STA and LTA values
    STA[0] = alpha_sta * np.sum(np.abs(x1[0:q_sta]))
    LTA[0] = alpha_lta * np.sum(np.abs(x1[0:q_lta]))
    
    # loop over time series
    j=1
    for i in range(q_lta+1,N1):
        
        STA[j] = STA[j-1] * (1 - alpha_sta) + alpha_sta * np.abs(x1[i])
        LTA[j] = LTA[j-1] * (1 - alpha_lta) + alpha_lta * np.abs(x1[i])
        
        j = j + 1
        
    # Compute characteristic function
    CF = STA/LTA
    
    # normalize characteristic function
    CF = CF / np.max(CF)
    
    if(kurtosis==True):
        
        # compute average, 2nd and 4th moment m, a2, a4
        
        # initialize arrays with zeros
        m = np.zeros(N)
        a2 = np.zeros(N)
        a4 = np.zeros(N)
    
        # arithmetic average m
        # --------------------
        
        # pad beginning of time series with time reversed time series
        x2 = np.concatenate((np.flipud(x[:q_k]), x), axis=0) 
        N2 = len(x2)
        
        # compute first m values
        m[0] = alpha_k * np.sum(x2[0:q_k], axis=0)
        
        # loop over time series
        j = 1
        for i in range(q_k+1,N2):
            
            m[j] = m[j-1] * (1 - alpha_k) + alpha_k * x2[i]
            
            j = j + 1
        
        # moments a2, a4
        # --------------
        
        # compute first a2, a4 values
        a2[0] = alpha_k * np.sum((x2[0:q_k]-m[0])**2)
        a4[0] = alpha_k * np.sum((x2[0:q_k]-m[0])**4)
        
        # loop over time series
        j = 1
        for i in range(q_k+1,N2):
  
            a2[j] = a2[j-1] * (1 - alpha_k) + alpha_k * (x2[i]-m[j])**2
            a4[j] = a4[j-1] * (1 - alpha_k) + alpha_k * (x2[i]-m[j])**4
            
            j = j + 1
    
        # compute Kurtosis
        epsk = 1e-3
        K = a4 / (a2**2+epsk)
        
        # normalize Kurtosis
        K = K / np.max(K) 
    
    # Initialize Plots
    plt.figure(figsize=(20,10))
    
    plt.subplot(311)
    
    # Plot time series
    plt.plot(t, x, 'b', label = r'time series x = xspike + xnoise')
    plt.ylabel('x(t) (s)')
    plt.legend()
    
    title = 'Time series x(t)'
    plt.title(title)
    
    plt.subplot(312)
    
    # Plot CF
    plt.plot(t, CF, 'c', label = r'Characteristic Function CF')
    
    if(kurtosis==True):
        plt.plot(t, K, 'm', label = r'Kurtosis K')
    
    plt.ylabel('CF(t) (s)')
    
    if(kurtosis==True):
        plt.ylabel('CF(t), K(t) (s)')
    
    plt.legend()
    
    title = 'CF'
    
    if(kurtosis==True):
        title = 'CF, K'
        
    plt.title(title)
    

    plt.subplot(313)
    
    # Plot STA, LTA
    plt.plot(t, STA, 'r', label = r'STA')
    plt.plot(t, LTA, 'g', label = r'LTA')
    plt.xlabel('Time (s)')
    plt.ylabel('STA(t), LTA(t) (s)')
    plt.legend()
    
    title = 'STA and LTA'
    plt.title(title)

In [4]:
interactive_plot = interactive(stalta_kurtosis, sn=(.01, 1, .01), wsta=(1, 50, 1), wlta=(1, 400, 1), wk=(1, 400, 1), kurtosis=False, wavelet=False)
output = interactive_plot.children[-1]
output.layout.height = '380px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='sn', max=1.0, min=0.01, step=0.01), IntSlider(value=…